In [ ]:
import json
import torch
from mlx_lm.utils import load
from mlx_lm.generate import generate
from Phase_1_Scripts_and_Notebooks.Phase_1_few_shots import few_shots

In [2]:
def parse_and_print_json(response):
    try:
        json_response = json.loads(response)
        print("Parsed JSON Successfully!")
        for key, value in json_response.items():
            if isinstance(value, list):
                print(f"{key}:")
                for item in value:
                    print(f" - {item}")
            else:
                print(f"{key}:\n - {value}")
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", e)

In [ ]:
model, tokenizer = load(path_or_hf_repo="Phase_1_MLX_Fine_Tuned")

In [50]:
system_prompt = """
You are an expert code analyzer that outputs ONLY valid and parsable JSON. Strictly follow these rules:
1. Analyze the provided code completely, but ONLY for top-level components
2. Output MUST be ONLY the JSON object with no extra text, formatting, or characters
3. Use this exact structure (including only components that exist in the code):
{
    "programming_language": "<C|JAVA|PYTHON|SQL|C#|JAVASCRIPT|RUBY|TYPESCRIPT|C++|KOTLIN|RUST|GO|PHP|SCALA>",
    "components": [
      {
        "component_type": "<IMPORT_STATEMENT|INCLUDE_STATEMENT|USING_STATEMENT|FUNCTION_DEFINITION|CLASS_DEFINITION|METHOD_DEFINITION|INTERFACE_DEFINITION|STRUCT_DEFINITION|ENUM_DEFINITION|GLOBAL_CODE|MODULE_DEFINITION|PACKAGE_DEFINITION|SCHEMA_DEFINITION|PROCEDURE_DEFINITION|TRIGGER_DEFINITION|MACRO_DEFINITION>",
        "component_name": "<name>",
        "component_code": "<complete_top_level_code_segment>",
        "component_description": "<technical_description>"
      }
    ],
    "overall_description": "<complete_code_summary>"
}

Requirements:
- ONLY analyze top-level components (do NOT break down internal implementation details)
- For functions/classes/methods: include the ENTIRE definition (including all nested code)
- Never split components into internal blocks (loops, conditionals, variable declarations)
- Only include the component types listed above
- Skip all other component types completely
- Never include null or empty fields
- Escape all special characters in JSON values properly
- Include complete top-level code segments exactly as they appear
- Provide detailed technical descriptions for each extracted component
- Describe overall functionality and architecture concisely
- Ensure the output is valid JSON that can be parsed directly
- Never add comments, notes, or text outside the JSON structure
- Never use markdown formatting or code blocks
- Maintain consistent JSON structure with proper escaping
- Only output the raw JSON object with no additional text
"""

In [ ]:
with open("Test_Code_Files/GradebookApp.java", "r") as f:
    code = f.read()

In [52]:
few_shots_prompt = []

count = 0

for k, v in few_shots.items():
    few_shots_prompt.append({"role": "user", "content": v[0].strip()})
    few_shots_prompt.append({"role": "assistant", "content": v[1].strip()})
    count += len(v[0].strip()) + len(v[1].strip())

In [53]:
messages = []
messages.append({"role": "assistant", "content": system_prompt.strip()})
messages += few_shots_prompt
messages.append({"role": "user", "content": code.strip()})

In [54]:
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True)

In [55]:
# Assistant response
response = generate(
    model,
    tokenizer,
    prompt=prompt,
    max_tokens=3072
)

In [56]:
response

'{\n  "programming_language": "JAVA",\n  "components": [\n    {\n      "component_type": "CLASS_DEFINITION",\n      "component_name": "Student",\n      "component_code": "public class Student {\\n    private int id;\\n    private String name;\\n    private Map<String, Double> grades;\\n\\n    public Student(int id, String name) {\\n        this.id = id;\\n        this.name = name;\\n        this.grades = new HashMap<>();\\n    }\\n\\n    public int getId() { return id; }\\n    public String getName() { return name; }\\n\\n    public void addGrade(String subject, double grade) {\\n        if (grade < 0 || grade > 100) {\\n            System.out.println(\\"Invalid grade for \\" + subject + \\". Must be between 0 and 100.\\");\\n            return;\\n        }\\n        grades.put(subject, grade);\\n    }\\n\\n    public Double getGrade(String subject) {\\n        return grades.get(subject);\\n    }\\n\\n    public double getAverage() {\\n        return grades.values().stream().mapToDoubl

In [57]:
parse_and_print_json(response)

Parsed JSON Successfully!
programming_language:
 - JAVA
components:
 - {'component_type': 'CLASS_DEFINITION', 'component_name': 'Student', 'component_code': 'public class Student {\n    private int id;\n    private String name;\n    private Map<String, Double> grades;\n\n    public Student(int id, String name) {\n        this.id = id;\n        this.name = name;\n        this.grades = new HashMap<>();\n    }\n\n    public int getId() { return id; }\n    public String getName() { return name; }\n\n    public void addGrade(String subject, double grade) {\n        if (grade < 0 || grade > 100) {\n            System.out.println("Invalid grade for " + subject + ". Must be between 0 and 100.");\n            return;\n        }\n        grades.put(subject, grade);\n    }\n\n    public Double getGrade(String subject) {\n        return grades.get(subject);\n    }\n\n    public double getAverage() {\n        return grades.values().stream().mapToDouble(Double::doubleValue).average().orElse(0.0);\n 